### Building Fast Queries on a CSV

In this project, we will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

We will use the laptops.csv file as our inventory. This CSV file was adapted from the __[Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices)__. 

The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

Given a laptop id, find the corresponding data.
Given an amount of money, find whether there are two laptops whose total price is that given amount.
Identify all laptops whose price falls within a given budget.

To begin, we read in our data

In [1]:
from csv import reader
    
with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/laptops.csv", encoding = "utf-8") as file:
    read = list(reader(file))
    header = read[0]
    rows = read[1:]
print(header)
print(rows[:5])
print(len(rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

__Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in it.__

__Throughout this project, we will make several improvements to the Inventory class. We will copy and paste the last version of the class, then add the new functionalitiees we want to implement. The new functionalities for every duplicated version will be marked _(#New Change)_. This will help you keep track of the changes that we make.__

In [2]:
class Inventory():
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])

In [3]:
Test = Inventory("laptops.csv")
print(Test.header)
print(len(Test.rows))
print(Test.rows[:2])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', 898]]


__The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.__


__For this, we will write a function named get_laptop_from_id(). This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.__

In [4]:
class Inventory():
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
    def get_laptop_from_id(self, laptop_id):    #New Change
        for row in self.rows:        #New Change
            if row[0] == laptop_id:          #New Change
                return row                  #New Change
        return "None"

In [5]:
Test1 = Inventory("laptops.csv")
print(Test1.get_laptop_from_id('3362737'))
print(Test1.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


__The algorithm from the previous screen requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity `O(R)` where R is the number of rows.__

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, we imagine that this code could be used in situations where the inventory contains millions of rows. Also, if we perform a lot of queries, even on a small dataset, the slow query performance will start to add up. It might eventually become the bottleneck of the application.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the `get_laptop_from_id()` method. We can do this by:

- Preprocess the data and create the dictionary in the `__init__()` method.
- Re-implement the `get_laptop_from_id()` method. We will do it as a new method so that we can compare the two.

In [6]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}            #New Change
        for row in self.rows:        #New Change
            self.id_to_row[row[0]] = row          #New Change
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return "None"
    
    def get_laptop_from_id_fast(self, laptop_id):         #New Change
        if laptop_id in self.id_to_row:                  #New Change
            return self.id_to_row[laptop_id]         #New Change
        else:                             #New change
            return "None"               #New Change

In [7]:
Test1 = Inventory("laptops.csv")
print(Test1.get_laptop_from_id('3362737'))
print(Test1.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


__Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs. We will use the time module to measure the execution time of each lookup and, for each method, add all times together.__

In [8]:
import time
import random

ids = [str(random.randint(1000000,9999999)) for j in range(10000)]
Test2 = Inventory("laptops.csv")

total_time_no_dict = 0
for x in ids:
    start = time.time()
    Test2.get_laptop_from_id(x)
    stop = time.time()
    total_time_no_dict += stop-start
    
total_time_dict = 0
for x in ids:
    start = time.time()
    Test2.get_laptop_from_id_fast(x)
    stop = time.time()
    total_time_dict += stop-start

print(total_time_no_dict)
print(total_time_dict)

1.8743224143981934
0.0


__We can see that the method that prepeocesses the data into a dictionary rus way faster than the initial method which loops through all the rows to search for the laptop_id__

__Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.__

__For example,__ imagine that there are only three laptops in inventory:

The prices of these laptops are USD 1,339, USD 898, and USD 575. Say we offered a gift card of USD 2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is USD 2,237 (USD 1,339 plus USD 898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full USD 2,500.

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

In [9]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}           
        for row in self.rows:        
            self.id_to_row[row[0]] = row          
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return "None"
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return "None"
    def check_promotion_dollars(self, dollars):         #New Change
        for i in self.rows:                 #New Change
            for j in self.rows:        #New Change
                if dollars == i[-1] or dollars == i[-1] + i[-1]:       #New Change
                    return True
        return False           #New Change

In [10]:
Test3 = Inventory("laptops.csv")
print(Test3.check_promotion_dollars(1000))
print(Test3.check_promotion_dollars(442))

True
False


__Let's try to make our code run faster.__

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.


In [11]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:        
            self.id_to_row[row[0]] = row 
        self.prices = set()                        #New Change
        for row in self.rows:                         #New Change
            self.prices.add(row[-1])                  #New Change
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return "None"
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return "None"
    def check_promotion_dollars(self, dollars):    
        for i in self.rows:          
            for j in self.rows:
                if dollars == i[-1] or dollars == i[-1] + i[-1]:
                    return True
        return False      
    
    def check_promotion_dollars_fast(self, dollars):      #New Change
        for i in self.prices:                            #New Change
            for j in self.prices:                        #New Change
                if i == dollars or i+j == dollars:                 #New Change
                    return True                        #New Change
        return False                                   #New Change

        

In [12]:
Test4 = Inventory("laptops.csv")
print(Test4.check_promotion_dollars_fast(1000))
print(Test4.check_promotion_dollars_fast(442))

True
False


In [13]:
prices = [str(random.randint(100,5000)) for j in range(100)]
Test5 = Inventory("laptops.csv")

total_time_no_set = 0
for x in prices:
    start = time.time()
    Test5.check_promotion_dollars(x)
    stop = time.time()
    total_time_no_set += stop-start
    
total_time_set = 0
for x in prices:
    start = time.time()
    Test5.check_promotion_dollars_fast(x)
    stop = time.time()
    total_time_set += stop-start

print(total_time_no_set)
print(total_time_set)

52.965383529663086
10.139368772506714


__In this project, we are going to leverage and extend that algorithm to help a customer find all laptops that fall within their budget.__

More formally, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars:

In [14]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open ("C:/Users/kenechukwu.ifediorah/OneDrive - EverCare (1)/Documents/PythonDA/Datasets/{name}".format(name = csv_filename), encoding = "utf-8") as file:
            read = list(reader(file))
            self.header = read[0]
            self.rows = read[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:        
            self.id_to_row[row[0]] = row 
        self.prices = set()                       
        for row in self.rows:                        
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key = lambda row : row[-1])          #New Change
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return "None"
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return "None"
    def check_promotion_dollars(self, dollars):    
        for i in self.rows:          
            for j in self.rows:
                if dollars == i[-1] or dollars == i[-1] + i[-1]:
                    return True
        return False      
    
    def check_promotion_dollars_fast(self, dollars):     
        for i in self.prices:                            
            for j in self.prices:                        
                if i  == dollars or i+j == dollars:                
                    return True                       
        return False                                   
        
    def find_first_laptop_more_expensive(self, price):   #(The entire function is a new change)
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_start + range_end) // 2
            current_price = self.rows_by_price[range_middle][-1]
            if current_price > price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        current_price = self.rows_by_price[range_start][-1]
        if current_price <= price:                  
            return -1                                      
        return range_start
    
        

In [15]:
Test6 = Inventory("laptops.csv")
print(Test6.find_first_laptop_more_expensive(1000))
print(Test6.find_first_laptop_more_expensive(10000))

683
-1


### Conclusion
**In this project, three functionalities were created:**

- Looking up laptops by their ID number
- Seeing how many laptops a customer could afford given their budget
- Determining the highest priced laptop at which the customer could not afford; determining all of the laptops within the customer's budget

**The three built-in Python modules used were: csv, time, and random**

**In this project, data pre-processing was used to significantly improve the time efficiency of the methods.**